In [ ]:
# U-Net with ResNet50 Encoder for Golf Course Segmentation
# Image size: 832x512, Classes: 6

# Check environment
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab")
    !pip install -q kagglehub
else:
    print("Running locally")

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
import kagglehub

In [ ]:
# GPU Configuration
print(f"TensorFlow version: {tf.__version__}")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    keras.mixed_precision.set_global_policy('mixed_float16')
    print(f"GPU configured: {len(gpus)} device(s)")
    for gpu in gpus:
        print(f"  - {gpu}")
else:
    print("No GPU detected, using CPU")

In [ ]:
# Download dataset (works on both Colab and local)
# Note: You may need to authenticate with Kaggle on first run
# For Colab: upload kaggle.json or use kagglehub.login()

dataset_path = kagglehub.dataset_download('jacotaco/danish-golf-courses-orthophotos')
print(f"Dataset path: {dataset_path}")

In [ ]:
# Hyperparameters
BATCH_SIZE = 2  # Reduce to 1 if OOM on Colab
IMAGE_SIZE = (512, 832)  # Height x Width
IN_CHANNELS = 3
LEARNING_RATE = 1e-4
NUM_CLASSES = 6  # Background, Fairway, Green, Tee, Bunker, Water
MAX_EPOCHS = 10
AUGMENTATION_PROBABILITY = 0.25

# Dataset paths
base_path = dataset_path
IMAGES_DIR = os.path.join(base_path, '1. orthophotos')
SEGMASKS_DIR = os.path.join(base_path, '2. segmentation masks')
LABELMASKS_DIR = os.path.join(base_path, '3. class masks')

# Output directory
OUTPUT_DIR = '/content/output' if IN_COLAB else './output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Preview sample
orthophoto_list = os.listdir(IMAGES_DIR)
print(f"Total images: {len(orthophoto_list)}")

idx = 5
golf_image = Image.open(os.path.join(IMAGES_DIR, orthophoto_list[idx]))
golf_segmask = Image.open(os.path.join(SEGMASKS_DIR, orthophoto_list[idx].replace(".jpg", ".png")))

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].set_title('Orthophoto')
axes[1].set_title('Segmentation Mask')
axes[0].imshow(golf_image)
axes[1].imshow(golf_segmask)
plt.show()

In [ ]:
def load_and_preprocess_image(image_path, mask_path):
    """Load and preprocess image and mask pair."""
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, IMAGE_SIZE, method='nearest')
    mask = tf.cast(mask, tf.float32)
    mask = tf.squeeze(mask, axis=-1)

    return image, mask


def augment_image_and_mask(image, mask):
    """Apply synchronized augmentation to image and mask."""
    def apply_augmentation():
        mask_expanded = tf.expand_dims(mask, axis=-1)
        combined = tf.concat([image, mask_expanded], axis=-1)
        combined = tf.image.random_flip_left_right(combined)
        aug_image = combined[:, :, :3]
        aug_mask = combined[:, :, 3]
        aug_image = tf.image.random_brightness(aug_image, 0.1)
        aug_image = tf.image.random_contrast(aug_image, 0.9, 1.1)
        aug_image = tf.clip_by_value(aug_image, 0.0, 1.0)
        return tf.cast(aug_image, tf.float32), aug_mask

    def keep_original():
        return tf.cast(image, tf.float32), mask

    should_augment = tf.random.uniform([]) < AUGMENTATION_PROBABILITY
    return tf.cond(should_augment, apply_augmentation, keep_original)


def create_dataset(images_dir, labelmasks_dir, shuffle=True):
    """Create TensorFlow dataset from directories."""
    image_filenames = sorted(os.listdir(images_dir))
    image_paths = [os.path.join(images_dir, fname) for fname in image_filenames]
    mask_paths = [os.path.join(labelmasks_dir, fname.replace('.jpg', '.png')) for fname in image_filenames]

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths), seed=42)
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset, len(image_paths)


def prepare_datasets():
    """Prepare train/val/test datasets with 70/20/10 split."""
    full_dataset, total_size = create_dataset(IMAGES_DIR, LABELMASKS_DIR, shuffle=True)

    train_size = int(0.7 * total_size)
    val_size = int(0.2 * total_size)
    test_size = total_size - train_size - val_size

    print(f"Split: {train_size} train, {val_size} val, {test_size} test")

    train_ds = full_dataset.take(train_size)
    remaining = full_dataset.skip(train_size)
    val_ds = remaining.take(val_size)
    test_ds = remaining.skip(val_size)

    train_ds = train_ds.map(augment_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return train_ds, val_ds, test_ds

In [ ]:
def build_unet_resnet50(input_shape=(512, 832, 3), num_classes=6):
    """U-Net with ResNet50 encoder (ImageNet pretrained)."""
    inputs = keras.Input(shape=input_shape)

    # Encoder: ResNet50
    base_model = keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_tensor=inputs
    )

    # Skip connections at different resolutions
    skip_layer_names = [
        'conv1_relu',        # 1/2
        'conv2_block3_out',  # 1/4
        'conv3_block4_out',  # 1/8
        'conv4_block6_out',  # 1/16
    ]
    skip_connections = [base_model.get_layer(name).output for name in skip_layer_names]
    bottleneck = base_model.get_layer('conv5_block3_out').output  # 1/32

    # Decoder
    x = layers.Conv2DTranspose(512, kernel_size=2, strides=2, padding='same')(bottleneck)
    x = layers.Concatenate()([x, skip_connections[3]])
    x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)

    x = layers.Conv2DTranspose(256, kernel_size=2, strides=2, padding='same')(x)
    x = layers.Concatenate()([x, skip_connections[2]])
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)

    x = layers.Conv2DTranspose(128, kernel_size=2, strides=2, padding='same')(x)
    x = layers.Concatenate()([x, skip_connections[1]])
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)

    x = layers.Conv2DTranspose(64, kernel_size=2, strides=2, padding='same')(x)
    x = layers.Concatenate()([x, skip_connections[0]])
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)

    x = layers.Conv2DTranspose(32, kernel_size=2, strides=2, padding='same')(x)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)

    # Output
    outputs = layers.Conv2D(num_classes, kernel_size=1, padding='same', dtype='float32')(x)

    return keras.Model(inputs=inputs, outputs=outputs, name='UNet_ResNet50')

In [ ]:
# Build and compile model
model = build_unet_resnet50(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES)

model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Prepare datasets
train_ds, val_ds, test_ds = prepare_datasets()

In [ ]:
# Callbacks
callback_list = [
    callbacks.ModelCheckpoint(
        filepath=os.path.join(OUTPUT_DIR, 'best_unet_resnet50.keras'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        verbose=1,
        min_lr=1e-7
    )
]

In [ ]:
# Train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=MAX_EPOCHS,
    callbacks=callback_list,
    verbose=1
)

In [ ]:
# Evaluate
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Class colors for visualization
CLASS_COLORS = np.array([
    [0, 0, 0],        # Background
    [0, 140, 0],      # Fairway
    [0, 255, 0],      # Green
    [255, 0, 0],      # Tee
    [217, 230, 122],  # Bunker
    [7, 15, 247]      # Water
], dtype=np.float32) / 255.0

CLASS_NAMES = ['Background', 'Fairway', 'Green', 'Tee', 'Bunker', 'Water']


def mask_to_rgb(mask):
    """Convert class mask to RGB."""
    h, w = mask.shape
    rgb_mask = np.zeros((h, w, 3), dtype=np.float32)
    for class_id in range(NUM_CLASSES):
        rgb_mask[mask == class_id] = CLASS_COLORS[class_id]
    return rgb_mask

In [ ]:
# Visualize predictions
for images, masks in test_ds.take(3):
    predictions = model.predict(images, verbose=0)
    pred_masks = np.argmax(predictions, axis=-1)
    
    for i in range(min(2, images.shape[0])):
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(images[i].numpy())
        axes[0].set_title('Input')
        axes[0].axis('off')
        
        axes[1].imshow(mask_to_rgb(masks[i].numpy().astype(np.int32)))
        axes[1].set_title('Ground Truth')
        axes[1].axis('off')
        
        axes[2].imshow(mask_to_rgb(pred_masks[i]))
        axes[2].set_title('Prediction')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()

In [ ]:
# Save model
model.save(os.path.join(OUTPUT_DIR, 'final_unet_resnet50.keras'))
print(f"Model saved to {OUTPUT_DIR}")

In [ ]:
# Training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_history.png'), dpi=150)
plt.show()

In [ ]:
# Download model from Colab
if IN_COLAB:
    from google.colab import files
    files.download(os.path.join(OUTPUT_DIR, 'final_unet_resnet50.keras'))